import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


reading data from csv file 

In [2]:
df=pd.read_csv("adult.data.csv")
df.columns=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                  'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
pd.set_option('display.max_columns', None)

cleaning data

In [3]:
df["workclass"].replace(' ?',np.nan,inplace=True)
df['occupation'].replace(' ?',np.nan,inplace=True)
df['native-country'].replace(' ?',np.nan,inplace=True)
df.drop_duplicates(inplace=True)
df.dropna(inplace =True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_9592\1610863650.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["workclass"].replace(' ?',np.nan,inplace=True)
C:\Users\Dell\AppData\Local\Temp\ipykernel_9592\1610863650.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [ ]:
df.head(15)

In [4]:

Y = df['income']
Y.index=np.arange(len(Y))
X=(df.drop('income',axis=1))
X.index=np.arange(len(X))

In [5]:
class NaiveBayes_Classifier:
    post_prob_less50=[random.random for i in range(len(Y))]
    post_prob_over50=[random.random  for i in range(len(Y))]  
    predict=pd.Series([random.random for i in range(len(Y))])
    mtrx=pd.DataFrame({})

    def __init__(self,X,Y) :
       self.X=X
       self.Y=Y
      
    
    def priors(self):
     prior_prob={}
    # calculate prior probability for each class
     for indx,y in enumerate(Y.unique()):
      l=X[y==Y] # l is to get all rows with same class (<=50 or >50)
      prior_prob[y]=l.shape[0]/len(X)
  
     return prior_prob
    # calculate likelihood of each feature for certain row
    def cond_prob(self,col,val,l):
     flag=False
     # since we want to get conditional prob for each class 
     # so there are some values of features exist in only one class 
     # so the conditional prob for this feature given anothor class is zero
     # such as 'private' in 'workclass' column if all rows that have private in workclass attribute 
     #belong to <=50 class then p(private|>50)=0++
     for i in l[col].unique():
        if( val ==i):
            flag=True
     if flag==True: 
      return l[col].value_counts(normalize=True)[val]
     else:
      return 0   

    # this function is to loop over the columns(features) of certain row and 
    #calculate cond_prob of each feature in this row  
    def _product(self,row):
     posterior={}
     pro=np.zeros(X.shape[1])
     for y in Y.unique():
       l=X[y==Y]# l is data with same class
       for indx,x in enumerate(l):# x is for column in l
         pro[indx]= self.cond_prob(x,X.loc[row,x],l)
       posterior[y]=self.priors()[y]*np.prod(pro)
     # it returns dict with keys(<=50,>50) and values of keys are posterior probs 
     #(<=50|attribute of row) (>50|attribute of row)
     return posterior 
    
    # this function makes prediction based on the max of the posteriors
    def prediction(self):
      predicted=[random.random for i in range(len(Y))]
      t={}
      for indx,i in enumerate(Y.index):
        t=self._product(i)
        self.post_prob_less50[i]=(t[Y.unique()[0]])
        self.post_prob_over50[i]=(t[Y.unique()[1]])
        
        predicted[i]=max(t,key=t.get)#it returns the key of the max of the two
      p=pd.Series(predicted )
      self.predict=p
      return p
    
    def _confusionMatrix(self):
      TPCount=0
      TNCount=0
      FPCount=0
      FNCount=0

      for x in range(len(Y)):
           if(Y.iloc[x]==self.predict.iloc[x] and Y.iloc[x]==Y.unique()[0]):
             TPCount+=1 #test and actual <=50
           elif(Y.iloc[x]==self.predict.iloc[x] and Y.iloc[x]==Y.unique()[1]):
               TNCount+=1 #test and actual >50
           elif(Y.iloc[x]!=self.predict.iloc[x] and Y.iloc[x]==Y.unique()[0]):
              # print(Y.iloc[x],"actual","\n test ",p.iloc[x])
               FPCount+=1 #test >50 and actual <=50
           else:
               FNCount+=1
      print('TP',TPCount)  
      print('TP',TPCount/len(Y))         
      mtrx=pd.DataFrame({'<=50':[TPCount/len(Y),FNCount/len(Y)],'>50':[FPCount/len(Y),TNCount/len(Y)]})
      indxName=pd.DataFrame({"pred|act":["test <=50","test >50"]})
      mtrx=pd.concat([mtrx,indxName], axis=1)
      mtrx.set_index("pred|act",inplace=True)
      self.mtrx=mtrx     
      return mtrx
    
    def sensitivity(self):
    # it returns TP/TP+FN
     return self.mtrx.loc['test <=50','<=50']/(self.mtrx.loc['test <=50','<=50']+self.mtrx.loc['test >50','<=50'])

    def specificity(self):
    #it returns TN/TN+FP
     return self.mtrx.loc['test >50','>50']/(self.mtrx.loc['test >50','>50']+self.mtrx.loc['test <=50','>50'])
        
      

In [6]:
nbc=NaiveBayes_Classifier(X,Y)


In [7]:
nbc.priors()

{' <=50K': 0.7509456500099542, ' >50K': 0.24905434999004578}

In [85]:
for i in Y.unique():
    print(i)
    print(nbc.cond_prob('age',31,X[i==Y]))

 <=50K
0.029780841286673736
 >50K
0.02358113509192646


In [10]:
print(nbc._product(7))
Y.iloc[7]

{' <=50K': 0.0, ' >50K': 1.2214830032905879e-15}


' >50K'

In [8]:
p=nbc.prediction()

In [9]:
p.value_counts()

<=50K    21674
>50K      8464
Name: count, dtype: int64

In [10]:
Y.value_counts()

income
<=50K    22632
>50K      7506
Name: count, dtype: int64

In [11]:
df["post_prob<=50"]=nbc.post_prob_less50
df["post_prob>50"]=nbc.post_prob_over50
df["predicted"]=p

In [12]:
df.head(16)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,post_prob<=50,post_prob>50,predicted
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,1.763863e-15,1.934379e-13,>50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,4.694552e-11,0.000000e+00,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,8.941740e-14,0.000000e+00,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,7.573557e-16,0.000000e+00,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,7.692732e-14,0.000000e+00,<=50K
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K,2.360225e-19,5.130866e-25,<=50K
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K,0.000000e+00,2.718900e-11,>50K
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K,0.000000e+00,1.221483e-15,>50K
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K,0.000000e+00,3.272955e-11,>50K
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K,0.000000e+00,8.253443e-13,>50K


In [13]:
nbc._confusionMatrix()

TP 21280
TP 0.7060853407658106


,<=50,>50
pred|act,,
test <=50,0.706085,0.044860
test >50,0.013073,0.235981


In [14]:
print("sensitivity : ",nbc.sensitivity())
print("specificity : ",nbc.specificity())

sensitivity :  0.9818215373258282
specificity :  0.8402646502835539


In [15]:
m=nbc._confusionMatrix()
m.loc['test <=50','<=50']=str(m.loc['test <=50','<=50'])+' (TP)'
m.loc['test <=50','>50']=str(m.loc['test <=50','>50'])+' (FP)'
m.loc['test >50','<=50']=str(m.loc['test >50','<=50'])+' (FN)'
m.loc['test >50','>50']=str(m.loc['test >50','>50'])+' (TN)'
print(m)

TP 21280
TP 0.7060853407658106
                                <=50                       >50
pred|act                                                      
test <=50    0.7060853407658106 (TP)  0.04486030924414361 (FP)
test >50   0.013073196628840666 (FN)  0.23598115336120512 (TN)


C:\Users\Dell\AppData\Local\Temp\ipykernel_9592\4082071067.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7060853407658106 (TP)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  m.loc['test <=50','<=50']=str(m.loc['test <=50','<=50'])+' (TP)'
C:\Users\Dell\AppData\Local\Temp\ipykernel_9592\4082071067.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.04486030924414361 (FP)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  m.loc['test <=50','>50']=str(m.loc['test <=50','>50'])+' (FP)'


## Sample of dataset to try code sice the whole dataset take a lot of time

In [16]:
Y = df['income'].head(3000)
Y.index=np.arange(len(Y))
X=(df.drop(['income','post_prob<=50','post_prob>50','predicted'],axis=1)).head(3000)
X.index=np.arange(len(X))

In [17]:
X.head(10)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States


In [25]:
nbc2=NaiveBayes_Classifier(X,Y)

In [26]:
nbc2.priors()

{' <=50K': 0.741, ' >50K': 0.259}

In [27]:
for i in Y.unique():
    print(i)
    print(nbc2.cond_prob('age',31,X[i==Y]))

 <=50K
0.03733693207377418
 >50K
0.021879021879021878


In [23]:
print(nbc2._product(7))
Y.iloc[7]

{' <=50K': 0.0, ' >50K': 8.907384878887418e-15}


' >50K'

In [28]:
p2=nbc2.prediction()

In [29]:
p2.value_counts()

<=50K    2209
>50K      791
Name: count, dtype: int64

In [30]:
Y.value_counts()

income
<=50K    2223
>50K      777
Name: count, dtype: int64

In [40]:
X["post_prob<=50"]=nbc2.post_prob_less50[:3000]
X["post_prob>50"]=nbc2.post_prob_over50[:3000]
X["predicted"]=p2
X['income']=Y

In [41]:
X.head(3000)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,post_prob<=50,post_prob>50,predicted,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,2.029055e-14,0.000000e+00,<=50K,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,4.736548e-10,0.000000e+00,<=50K,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1.202480e-12,0.000000e+00,<=50K,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1.151576e-14,0.000000e+00,<=50K,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,7.443934e-13,0.000000e+00,<=50K,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,30,Private,186269,Bachelors,13,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States,7.148911e-11,0.000000e+00,<=50K,<=50K
2996,37,Private,286026,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,0.000000e+00,1.804789e-08,>50K,>50K
2997,30,Private,403433,Bachelors,13,Never-married,Prof-specialty,Not-in-family,Black,Male,0,0,50,United-States,0.000000e+00,2.979027e-12,>50K,>50K
2998,73,Private,123160,10th,6,Widowed,Other-service,Not-in-family,White,Female,0,0,10,United-States,3.092181e-15,0.000000e+00,<=50K,<=50K


In [42]:
nbc2._confusionMatrix()

TP 2206
TP 0.7353333333333333


,<=50,>50
pred|act,,
test <=50,0.735333,0.005667
test >50,0.001000,0.258000


In [45]:
print("sensitivity : ",nbc2.sensitivity())
print("specificity : ",nbc2.specificity())

sensitivity :  0.9986419194205522
specificity :  0.9785082174462706
